In [25]:
# Libraries and and global variables
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers

K = None

In [26]:
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#Loading dataset for classification, scale and split test and train
iris = datasets.load_iris()
# Filter and conserve tipe 0 and 2 of iris
cond = (iris.target != 1)
# Preprocess dataset to be used in SVM
y = iris.target[cond] - 1
y = y.astype('float')
x = iris.data[cond].astype('float')
x = preprocessing.scale(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [27]:
sigma = 1

# Custom rbf kernel
def krbf(x,y,sigma):
    return np.exp(-np.linalg.norm(x - y)**2/(2*sigma**2))

# SVM with rbf kernel
def fitNonLinear(x, y): 
    global K
    NUM = x.shape[0]
    DIM = x.shape[1]
    
    Y = np.reshape(y, (NUM, 1))
    Ym = np.matmul(Y,Y.T)
    # we'll solve the dual
    # obtain the kernel
    K = np.zeros((NUM,NUM))
    for i in range(NUM):
        for j in range(NUM):
            K[i,j] = krbf(x[i], x[j], sigma)
    K = np.multiply(K, Ym)
    P = matrix(K)
    q = matrix(-np.ones((NUM, 1)))
    G = matrix(-np.eye(NUM))
    h = matrix(np.zeros(NUM))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    solvers.options['show_progress'] = False
    
    # Call to quadratic solver library
    sol = solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return alphas

In [28]:
# fit svm classifier
alphas = fitNonLinear(X_train, y_train)

# Get support vectors and b
cond = (alphas > 1e-4)
alpha_sv = alphas[cond]
n_sv = len(alpha_sv)
xCond = cond.reshape(-1)
x_sv = X_train[xCond, :]
y_sv = y_train[xCond]
K_sv = K[xCond,xCond]

b = np.mean(y_sv - K_sv*(np.multiply(alpha_sv, y_sv)))


In [29]:
ym = np.zeros(X_test.shape[0])
# Predict the test data
Ktest2 = []
for j in range(X_test.shape[0]):
    x_p = X_test[j]
    K_pred=np.zeros(x_sv.shape[0])
    for i in range(x_sv.shape[0]):
        K_pred[i] = krbf(x_p, x_sv[i], sigma)
    Ktest2.append(K_pred)
    ym[j] = np.sign(np.sum(np.multiply(np.multiply(alpha_sv,y_sv), K_pred)) + b)

In [30]:
from sklearn.metrics import confusion_matrix
#Create a confusion matrix to check the accuracy of the algorithm
confusion_matrix(y_test, ym)

array([[11,  0],
       [ 0, 14]], dtype=int64)

In [31]:
from sklearn import svm

skSVM = svm.SVC(gamma=0.5)
skSVM.fit(X_train, y_train)


confusion_matrix(y_test, skSVM.predict(X_test))

array([[11,  0],
       [ 0, 14]], dtype=int64)

In [34]:

skSVM.support_vectors_

array([[-0.10312784,  1.44055745, -0.86678372, -0.91514954],
       [-1.27261883,  0.95956999, -1.20256047, -1.02461719],
       [-0.74103202,  0.23808879, -0.86678372, -0.69621424],
       [-1.3789362 , -2.16684851, -1.05865615, -0.91514954],
       [-0.10312784,  2.88351983, -0.96271993, -0.80568189],
       [-0.84734938, -0.4833924 , -0.91475182, -1.02461719],
       [-1.59157092, -0.4833924 , -1.15459236, -1.13408484],
       [-0.63471466,  2.16203864, -0.96271993, -1.13408484],
       [ 2.02321941,  1.44055745,  1.53162169,  1.16473578],
       [ 0.42845897,  0.47858252,  0.90803629,  1.27420343],
       [ 0.00318952, -0.96437986,  0.76413196,  1.38367108],
       [ 0.21582425, -2.40734224,  0.71616386,  0.39846224],
       [ 1.4916326 , -0.4833924 ,  1.09990872,  0.50792989],
       [ 1.70426733, -0.96437986,  1.24381305,  0.83633283],
       [ 2.02321941, -0.4833924 ,  1.24381305,  1.27420343],
       [ 0.96004579, -1.68586105,  1.09990872,  0.72686519],
       [-0.2094452 , -0.

In [35]:
alpha_sv

array([0.42518836, 0.46459499, 0.20195655, 0.79411763, 0.56857673,
       0.12522993, 0.91242009, 0.8484252 , 0.25589565, 0.78605415,
       0.04651846, 0.34285701, 0.37819529, 0.39619176, 0.00727125,
       0.01133446, 0.42305865])

Comparing the scikit learn implementation and our implementation, we can see some major differences in the support vectors, this might be because the scikit learn uses a C-Support Vector Classification algorithm, scikit learn doesn't have the algorithm that we implemented, so we cannot do a direct comparison, this might be becuase scikit learn should use advanced algorithms to improve the time complexity of the excecution, in which our algorithm doesn't have good results